In [2]:
import pandas as pd
import numpy as np
datadir = '../../data/'



# Select correct number of EVs (person_ids) from each county to simulate

In [15]:
merged_df = pd.read_parquet(datadir+'wa_pop_and_trips_sorted_county.parquet', engine='pyarrow')

take a subset for speed

In [49]:
small = df.head(100000).copy()


In [50]:
county_list = small.destination_county.unique()

# make up a random number of people to select for each county 
# since I dont have the real numbers yet. 
people_to_select = [np.random.randint(1, 5) for _ in range(len(county_list))]

In [51]:
# make a data frame to mimic the output of the stock rollover model
ev_df = pd.DataFrame({ 'county': county_list, 'nEV': people_to_select, 'ef': [0.3]*len(county_list)})

In [65]:
def sample_people_by_county(df, ev_df, fraction=0.05):
    """ Selects a random sample of people (representing EVs) from each county.
    These numbers come from the stock rollover model.

    Args:
        df (_type_): _description_
        county (_type_): _description_
        num_to_select (_type_): _description_

    Returns:
        _type_: _description_
    """
    # get the unique people in the dataframe
    unique_df = df.drop_duplicates(subset=['person_id'])[['person_id', 'destination_county']]

    reduced_df = []
    for _, cnty in ev_df.iterrows():
        county = cnty['county']
        num_to_select = cnty['nEV']
        
        # slice the unique dataframe to only include the county
        county_df = unique_df[(unique_df['destination_county'] == county)]

        # make sure we don't select more people than are in the county
        if num_to_select > len(county_df):
            num_to_select = len(county_df)
            print(f'Warning: {num_to_select} people selected for {county} but only {len(county_df)} people in that {county}')

        if fraction is None:
            # unique people in that county
            selected = county_df.person_id.sample(n=num_to_select, replace=False, random_state=42)
        else:
            # unique people in that county
            selected = county_df.person_id.sample(frac=fraction, replace=False, random_state=42)
        
        # grab only those selected people from the original dataframe
        cnty_df = df[(df['person_id'].isin(selected))]

        # append it to the reduced dataframe
        reduced_df.append(cnty_df)

    final_df = pd.concat(reduced_df)
    return final_df

Try on the whole dataset

In [64]:
df = merged_df
county_list = df.destination_county.unique()
people_to_select = [np.random.randint(2, 100) for _ in range(len(county_list))]
# make a data frame to mimic the output of the stock rollover model
ev_df = pd.DataFrame({ 'county': county_list, 'nEV': people_to_select, 'ef': 0.3})

In [66]:
sub_df = sample_people_by_county(df, ev_df, fraction=0.05)